In [ ]:
# mcmc is for later
# from pyro.infer.mcmc import MCMC
armature = {"nothing": 0}

primitve_shapes = {
    "triangle": 3,
    "point": 1,
    "quad": 4,
    "triangle_strip": 3,
}

%load_ext autoreload
%autoreload 2

In [35]:
from functools import reduce


def compose_mesh(dx: list):
    print(dx)
    return dx


def is_blade_of_grass(mesh):
    print(mesh)
    if reduce(lambda x, y: x * y, map(lambda x: primitve_shapes[x], mesh)) < 12:
        return True
    else:
        return False


# this means that this will generate plolygons with proparty 2X as much
def reward(mesh):
    if is_blade_of_grass(mesh):
        return 10
    if "triangle_strip" in mesh:
        return 5


In [36]:
test_dx = ["triangle", "triangle_strip"]
reward(compose_mesh(test_dx))

['triangle', 'triangle_strip']
['triangle', 'triangle_strip']


10

# Flow Network
- The goal of the flow network will be to generate examples *proportional* to `reward`
- the net in GflowNet refers to an MDP
- 'input_dims'
    - For choosing in this `primitve_shapes` example there are $4$ *patches*
> not sure if `source` and`sink` have to be a defined composition. or can by any composition with a given property that it finds




In [24]:
import torch.nn as nn

"""
A neural network model class that inherits from `torch.nn.Module`.

Attributes:
-----------
mlp : torch.nn
    A multi-layer perceptron (MLP) neural network.

Methods:
--------
__init__(hid_dim: int) -> None
    Initializes the flowModdel class with a hidden dimension size.
"""


class FlowModel(nn.Module):
    def __init__(self, hid_dim, input_dim: int) -> None:
        super().__init__()
        # input_dim:int of possable actions
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, hid_dim), nn.LeakyReLU(), nn.Linear(hid_dim, input_dim)
        )
    def forward(self,x)->any:
        # flow must be positive o

        # unclear to me how x contains this information
        # ... 
        # and multiply by (1 - x) to give 0 flow to actions we know we can't take
        # (in this case, x[i] is 1 if a feature is already there, so we know we
        # can't add it again)
        # ...

        # in the paper this is F
        return self.mlp(x).exp() * (1-x)
    

$$\sum_{s: (s,s')\in\mathcal{E}} F(s,s') = R(s') + \sum_{s'':(s',s'')\in\mathcal{E}} F(s',s'')$$

In [18]:
from typing import List, Tuple

def face_parents(state: List[str]) -> Tuple[List[List[str]], List[int]]:
    """
    Find parent states and their corresponding actions for a given state.
    
    Args:
        state: Current state represented as a list of face parts
        
    Returns:
        Tuple of (parent_states, parent_actions) where:
        - parent_states is a list of states, each missing one face part
        - parent_actions is a list of indices corresponding to the removed parts
    """
    # Create parent states by removing one face part at a time
    parent_states = [[part for part in state if part != face_part] 
                    for face_part in state]
    
    # Sort the state to create sorted_keys
    sorted_keys = sorted(state)
    
    # Find the action index for each removed face part
    parent_actions = [sorted_keys.index(face_part) 
                     for face_part in state]
    
    return parent_states, parent_actions

# Train for Experiment

1. Set up `mlflow` : from `./notebooks` dir run
    ``` bash
    # im useing poatry so
    $ poetry run mlflow ui
    ```

In [ ]:
import os
from mlflow import log_metric, log_param, log_artifacts
import tqdm
import torch
from torch.optim import Adam
from torch.distributions.categorical import Categorical

# import duckdb
sorted_keys = primitve_shapes


def face_to_tensor(face):
    assert(False)
    return torch.tensor([i in face for i in primitve_shapes]).float()


# Instantiate model and optimizer
F_sa = FlowModel(hid_dim=512, input_dim=len(primitve_shapes))
opt = Adam(F_sa.parameters(), 3e-4)

# Let's keep track of the losses and the faces we sample
losses = []
sampled_faces = []
# To not complicate the code, I'll just accumulate losses here and take a
# gradient step every `update_freq` episode.
minibatch_loss = 0
update_freq = 4
for episode in tqdm.tqdm(range(50000), ncols=40):
    # Each episode starts with an "empty state"
    state = []
    # Predict F(s, a)
    edge_flow_prediction = F_sa(face_to_tensor(state))
    for t in range(3):
        # The policy is just normalizing, and gives us the probability of each action
        policy = edge_flow_prediction / edge_flow_prediction.sum()
        # Sample the action
        action = Categorical(probs=policy).sample()
        # "Go" to the next state
        print(f"Current state: {state},\n Sorted keys: {sorted_keys},\n State: {state}")
        new_state = state + [sorted_keys[action]]

        # Now we want to compute the loss, we'll first enumerate the parents
        parent_states, parent_actions = face_parents(new_state)
        # And compute the edge flows F(s, a) of each parent
        px = torch.stack([face_to_tensor(p) for p in parent_states])
        pa = torch.tensor(parent_actions).long()
        parent_edge_flow_preds = F_sa(px)[torch.arange(len(parent_states)), pa]
        # Now we need to compute the reward and F(s, a) of the current state,
        # which is currently `new_state`
        if t == 2:
            # If we've built a complete face, we're done, so the reward is > 0
            # (unless the face is invalid)
            reward = reward(new_state)
            # and since there are no children to this state F(s,a) = 0 \forall a
            edge_flow_prediction = torch.zeros(6)
        else:
            # Otherwise we keep going, and compute F(s, a)
            reward = 0
            edge_flow_prediction = F_sa(face_to_tensor(new_state))

        # The loss as per the equation above
        flow_mismatch = (
            parent_edge_flow_preds.sum() - edge_flow_prediction.sum() - reward
        ).pow(2)
        minibatch_loss += flow_mismatch  # Accumulate
        # Continue iterating
        state = new_state

    # We're done with the episode, add the face to the list, and if we are at an
    # update episode, take a gradient step.
    sampled_faces.append(state)
    if episode % update_freq == 0:
        losses.append(minibatch_loss.item())
        minibatch_loss.backward()
        opt.step()
        opt.zero_grad()
        minibatch_loss = 0


  0%|         | 0/50000 [00:00<?, ?it/s]

Current state: [],
 Sorted keys: {'triangle': 3, 'point': 1, 'quad': 4, 'triangle_strip': 3},
 State: []


KeyError: tensor(1)